In [1]:
import sys
!{sys.executable} -m pip install beautifulsoup4

import requests

from bs4 import BeautifulSoup

import pandas as pd

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Open the file and extract the elements of the table:

In [2]:
with open ('C:/Users/simserra/Documents/Zips.html') as html_file:
    Soup = BeautifulSoup(html_file, 'html.parser')

table = Soup.find('table', class_='wikitable sortable')

l = []
for point in Soup.find_all('table', class_ = 'wikitable sortable'):
        dpoint = table.tbody.text

l = dpoint.split('\n')

Read the soon-to-be columns into lists:

In [3]:
Postcode = []
for i in range(len(l)):
    if (i-1)%5 == 0:
        Postcode.append(l[i])
    else:
        pass

In [4]:
Borough = []
for i in range(len(l)):
    if (i-2)%5 == 0:
        Borough.append(l[i])
    else:
        pass

In [5]:
Neighborhood = []
for i in range(len(l)):
    if (i-3)%5 == 0:
        Neighborhood.append(l[i])
    else:
        pass

Create the dataframe, using the lists as columns:

In [6]:
df = pd.DataFrame(
    {
        "PostalCode": Postcode,
        "Borough": Borough,
        "Neighborhood": Neighborhood
    }
)

Where neighborhood is unassigned, name it by its borough:

In [7]:
for i in range(df.shape[0]):
    if df.Neighborhood[i] == 'Not assigned':
        df.Neighborhood[i] = df.Borough[i]
    else:
        pass

Drop rows with unassigned boroughs:

In [8]:
df = df[df.Borough != 'Not assigned']
df.head()

,Borough,Neighborhood,PostalCode
0,Borough,Neighbourhood,Postcode
3,North York,Parkwoods,M3A
4,North York,Victoria Village,M4A
5,Downtown Toronto,Harbourfront,M5A
6,Downtown Toronto,Regent Park,M5A


Create two new columns to help with neighborhood concatenation:

In [9]:
df['repeat'] = 0
df['include'] = 0
df.head()

,Borough,Neighborhood,PostalCode,repeat,include
0,Borough,Neighbourhood,Postcode,0,0
3,North York,Parkwoods,M3A,0,0
4,North York,Victoria Village,M4A,0,0
5,Downtown Toronto,Harbourfront,M5A,0,0
6,Downtown Toronto,Regent Park,M5A,0,0


Make the concatenation for like postal codes:

In [10]:
for i in range(df.shape[0]-1):
    if df.iloc[i,2] == df.iloc[i-1,2]:
        df.iloc[i,3] = df.iloc[i,1] + ", " + df.iloc[i-1,3]
    else:
        df.iloc[i,3] = df.iloc[i,1]
    if df.iloc[i,2] == df.iloc[i+1,2]:
        df.iloc[i,4] = 0
    else:
        df.iloc[i,4] = 1
        
df.iloc[df.shape[0]-1,4] = 1

df.head()

,Borough,Neighborhood,PostalCode,repeat,include
0,Borough,Neighbourhood,Postcode,Neighbourhood,1
3,North York,Parkwoods,M3A,Parkwoods,1
4,North York,Victoria Village,M4A,Victoria Village,1
5,Downtown Toronto,Harbourfront,M5A,Harbourfront,0
6,Downtown Toronto,Regent Park,M5A,"Regent Park, Harbourfront",1


In [11]:
df.iloc[df.shape[0]-1,3] = df.iloc[df.shape[0]-1,1] + ", " + df.iloc[df.shape[0]-2,3]

In [12]:
df.tail()

,Borough,Neighborhood,PostalCode,repeat,include
284,Etobicoke,Kingsway Park South West,M8Z,Kingsway Park South West,0
285,Etobicoke,Mimico NW,M8Z,"Mimico NW, Kingsway Park South West",0
286,Etobicoke,The Queensway West,M8Z,"The Queensway West, Mimico NW, Kingsway Park S...",0
287,Etobicoke,Royal York South West,M8Z,"Royal York South West, The Queensway West, Mim...",0
288,Etobicoke,South of Bloor,M8Z,"South of Bloor, Royal York South West, The Que...",1


Remove extraneous data post-concatenation:

In [13]:
df = df[df.include == 1]

In [14]:
df.head()

,Borough,Neighborhood,PostalCode,repeat,include
0,Borough,Neighbourhood,Postcode,Neighbourhood,1
3,North York,Parkwoods,M3A,Parkwoods,1
4,North York,Victoria Village,M4A,Victoria Village,1
6,Downtown Toronto,Regent Park,M5A,"Regent Park, Harbourfront",1
8,North York,Lawrence Manor,M6A,"Lawrence Manor, Lawrence Heights",1


In [15]:
df = df[1:]

In [16]:
df.head()

,Borough,Neighborhood,PostalCode,repeat,include
3,North York,Parkwoods,M3A,Parkwoods,1
4,North York,Victoria Village,M4A,Victoria Village,1
6,Downtown Toronto,Regent Park,M5A,"Regent Park, Harbourfront",1
8,North York,Lawrence Manor,M6A,"Lawrence Manor, Lawrence Heights",1
9,Queen's Park,Queen's Park,M7A,Queen's Park,1


In [17]:
df2 = df.reset_index()

In [18]:
df2.head()

,index,Borough,Neighborhood,PostalCode,repeat,include
0,3,North York,Parkwoods,M3A,Parkwoods,1
1,4,North York,Victoria Village,M4A,Victoria Village,1
2,6,Downtown Toronto,Regent Park,M5A,"Regent Park, Harbourfront",1
3,8,North York,Lawrence Manor,M6A,"Lawrence Manor, Lawrence Heights",1
4,9,Queen's Park,Queen's Park,M7A,Queen's Park,1


In [19]:
for i in range(df2.shape[0]):
    df2.Neighborhood[i] = df2.repeat[i]

C:\Python27\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
df2.head()

,index,Borough,Neighborhood,PostalCode,repeat,include
0,3,North York,Parkwoods,M3A,Parkwoods,1
1,4,North York,Victoria Village,M4A,Victoria Village,1
2,6,Downtown Toronto,"Regent Park, Harbourfront",M5A,"Regent Park, Harbourfront",1
3,8,North York,"Lawrence Manor, Lawrence Heights",M6A,"Lawrence Manor, Lawrence Heights",1
4,9,Queen's Park,Queen's Park,M7A,Queen's Park,1


Correct the order of the columns:

In [21]:
df3 = df2.iloc[0:df.shape[0],[3,1,2]]

In [22]:
df3

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [23]:
df3.shape

(103, 3)